In [1]:
import numpy as np
import pickle
import sys
import torch
sys.path.append("../src")
from preprocess import new_grs, create_tensordata_new, convert_to_Data

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
path_ictal = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/ictal_1.pickle"
path_preictal = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/preictal_1.pickle"
path_postical = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/postictal_1.pickle"

data_preictal = pickle.load(open(path_preictal, 'rb'))
data_ictal = pickle.load(open(path_ictal, 'rb'))
data_postictal = pickle.load(open(path_postical, 'rb'))

#### Structure
List with entries of the form `graph = [A, NF, EF]`. Where `A`, `NF`, and `EF` are lists of length 4, 3, and 4 respectively. 

`A = [A0, A1, A2, A3]`, where :
-   `A0` = Ones, shape `(107,107)`.
-   `A1` = Correlation, shape `(107,107)`.  
-   `A2` = Coherence, shape `(107,107)`.
-   `A3` = Phase, shape `(107,107)`.

`NF = [NF0, NF1, NF2]`, where:

-  `NF0` = Ones, shape `(107,1)`.
-  `NF1` = Average Energy, shape `(107,1)`.
-  `NF2` = Band Energy, shape `(107,8)`.


`EF = [EF0, EF1, EF2, EF3]`, where:

-  `EF0` = Ones, shape `(107,107,1)`.
-  `EF1` = Correlation, shape `(107,107,1)`.
-  `EF2` = Coherence, shape `(107,107,1)`.
-  `EF3` = Phase, shape `(107,107,1)`.

All the information above has been (experimentally) confirmed by the above and Alan's documentation of `get_nf`, `get_adj`, and `get_ef` helper functions in his `load_data()` function, but should talk to Alan about confirming these details for absolute certainty.

In [3]:
print(f"A0 = E0 for first graph? {np.array_equal(data[0][0][0], data[0][2][0].reshape(107, 107))}.")
print(f"A1 = E1 for first graph? {np.array_equal(data[0][0][1], data[0][2][1].reshape(107, 107))}.")
print(f"A2 = E2 for first graph? {np.array_equal(data[0][0][2], data[0][2][2].reshape(107, 107))}.")
print(f"A3 = E3 for first graph? {np.array_equal(data[0][0][3], data[0][2][3].reshape(107, 107))}.")

A0 = E0 for first graph? True.
A1 = E1 for first graph? True.
A2 = E2 for first graph? True.
A3 = E3 for first graph? True.


#### Temporal Concatenation
For each run, we are going to concatenate the features in a larger list of `[preictal, ictal, postictal]`. We're then going to feed this into the `preprocess.ipynb` pipeline we've created (or just using one big patch). For simplicity we're going to use the most extensive graph representation possible by using:
-  `A` = None
-  `NF` = Average Energy and Band Energy, shape `(107,9)`.
-  `EF` = Correlation, Coherence, Phase, shape `(107, 107, 3)`.

for each patient.

We'll call this new list of graph representations `new_data`.

In [3]:
from preprocess import new_grs

# Make new data for each class
new_data_preictal = new_grs(data_preictal, type="preictal", mode="binary")
new_data_ictal = new_grs(data_ictal, type="ictal", mode="binary")
new_data_postictal = new_grs(data_postictal, type="postictal", mode="binary")

# Concatenate all data temporally
new_data = new_data_preictal + new_data_ictal + new_data_postictal

Now we'll convert `new_data` to `pyg_data` so it is in tensor format.

We'll name this new list of torch tensors with entries `[[edge_index, x, edge_attr], y]` as `pyg_grs`.

In [4]:
pyg_grs = create_tensordata_new(num_nodes=107, data_list=new_data, complete=True, save=False, logdir=None)

In [2]:
import sys
import torch
sys.path.append("../src")
from preprocess import pseudo_data

pyg_grs = torch.load("/Users/xaviermootoo/Documents/VScode/ssl-seizure-detection/pyg_grs.pt")
pdata = pseudo_data(pyg_grs, tau_pos = 12 // 0.12, tau_neg = (75) // 0.12, stats = True, save = False, patientid = "patient", 
                    logdir = None, model = "temporal_shuffling", sample_ratio=0.10)

Number of examples: 455510
y
1    227755
0    227755
Name: count, dtype: int64


After this we then convert the list of tensor `pyg_grs` to the PyG data structure `torch_geometric.data.Data`.

In [12]:
Data_list = convert_to_Data(pyg_grs, save=False, logdir=None)

The rest is up to `preprocess.ipynb` to handle...

In [3]:
import sys
sys.path.append("../src")
import torch
from new_patch import patch

# Test case
path_ictal = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/ictal_1.pickle"
path_preictal = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/preictal_1.pickle"
path_postical = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/patient_gr/jh101/postictal_1.pickle"

graphrep_dir = [path_preictal, path_ictal, path_postical]
logdir = "/Users/xaviermootoo/Documents/Data/ssl-seizure-detection/pyg_data/jh101"
filename = "jh101_run1"

In [4]:
patched_data = patch(graphrep_dir=graphrep_dir,  logdir=logdir, file_name=filename, num_electrodes=107, tau_pos=12//0.12, tau_neg=(7 * 60)//0.12, 
          model="supervised", stats=True, sample_ratio=1.0)

In [5]:
patched_data_rp = patch(graphrep_dir=graphrep_dir,  logdir=logdir, file_name=filename, num_electrodes=107, tau_pos=12//0.12, tau_neg=(7 * 60)//0.12, 
          model="relative_positioning", stats=True, sample_ratio=1.0)

Number of examples: 125307
y
1    125307
Name: count, dtype: int64
